The source code is in the public domain and not licensed or under
copyright. The information and software may be used freely by the public.
As required by 17 U.S.C. 403, third parties producing copyrighted works
consisting predominantly of the material produced by U.S. government
agencies must provide notice with such work(s) identifying the U.S.
Government material incorporated and stating that such material is not
subject to copyright protection.

Derived works shall not identify themselves in a manner that implies an
endorsement by or an affiliation with the Naval Research Laboratory.

RECIPIENT BEARS ALL RISK RELATING TO QUALITY AND PERFORMANCE OF THE
SOFTWARE AND ANY RELATED MATERIALS, AND AGREES TO INDEMNIFY THE NAVAL
RESEARCH LABORATORY FOR ALL THIRD-PARTY CLAIMS RESULTING FROM THE ACTIONS
OF RECIPIENT IN THE USE OF THE SOFTWARE.

In [ ]:
import sys
sys.path.append('../modules')
import numpy as np
import matplotlib as mpl
import matplotlib.pyplot as plt
import glob
import json
from scipy import constants as C
import scipy.interpolate
from cycler import cycler
import warnings
import grid_tools as grid
import init
# The following line provides interactive plots, and requires the ipympl package.
#%matplotlib widget

lbl3 = [r'$\varrho$ (mm)',r'$\varphi$',r'$z$ (mm)']

base_diagnostic = '../out/test'
sim_list = glob.glob(base_diagnostic+'*_sim.json')
uppe_list = glob.glob(base_diagnostic+'*_uppe_wave*.npy')
para_list = glob.glob(base_diagnostic+'*_paraxial_wave*.npy')

# Units

if len(sim_list)==0:
    raise FileNotFoundError("no simulation metadata")
if len(sim_list)>1:
    warnings.warn("Multiple metadata files, normalization is based on " + sim_list[0])
with open(sim_list[0]) as f:
    sim_obj = json.load(f)
    mks_length = sim_obj['mks_length']

total_list = uppe_list + para_list
if len(uppe_list)>0 and len(para_list)==0:
    real_field = True
elif len(uppe_list)==0 and len(para_list)>0:
    real_field = False
else:
    raise RuntimeError('cannot isolate type of field in output directory')
nruns = len(total_list)

monochrome = (cycler('linewidth', [1.0,1.0,2.0,1.0,1.0]) + cycler('linestyle', ['-', '--','-',':', '-.']) )
mpl.rcParams['text.usetex'] = True
mpl.rcParams['font.size'] = 12
plt.rcParams['axes.prop_cycle'] = monochrome

def get_Lambda2(rn):
    '''This is the radial scale factor for power flow, which is also the square of the symmetrizing operator for the DHT
    
    :param np.ndarra rn: radial mesh nodes'''
    dr = rn[1] - rn[0]
    return np.pi*(rn+0.5*dr)**2 - np.pi*(rn-0.5*dr)**2

def get_cal_factor(A,wn,rn,U0,dw):
    '''Get calibration factor scaled to produce units of U0/dw.  The units of A,wn,rn can be anything,
    so long as they stay the same wherever the calibration is used.  The calibration factor works in
    either r-space or kr-space.
    
    :param np.ndarray A: vector potential in frequency domain
    :param np.ndarray wn: frequency nodes
    :param np.ndarray rn: radial nodes
    :param np.ndarray U0: total energy for calibration
    :param np.ndarray dw: frequency interval for calibration (can be rad/s, Hz, wavenumbers, whatever)'''
    Iw = get_spectral_intensity(A,wn,0,1)
    Pw = get_spectral_power(Iw,rn)
    return U0/dw/np.sum(Pw)

def get_freq_time_data(A,ext):
    dr = (ext[3] - ext[2])/A.shape[1]
    r_nodes = np.linspace(ext[2]+dr/2,ext[3]-dr/2,A.shape[1])
    if real_field:
        # Nodes are like [0,1,2,3] and walls are like [-0.5,0.5,1.5,2.5,3.5]
        # In these examples the user's requested upper bound would be 4 (it is thrown out)
        # Therefore element N/2+1 should be regarded as the central frequency
        dw = (ext[1] - ext[0])/A.shape[0]
        wc = 0.5*ext[0] + 0.5*(ext[1] + dw)
        tmax = 2*np.pi/dw
        w_nodes = np.linspace(ext[0]+dw/2,ext[1]-dw/2,A.shape[0])
    else:
        # Nodes are like [-2,-1,0,1] and walls are like [-2.5,-1.5,-.5,.5,1.5]
        dw = (ext[1] - ext[0])/A.shape[0]
        wc = 0.5*ext[0] + 0.5*(ext[1] + dw)
        tmax = 2*np.pi/dw
        w_nodes = np.linspace(ext[0]+dw/2,ext[1]-dw/2,A.shape[0])
    return dw,wc,tmax,w_nodes,r_nodes

def get_time_domain_intensity(A,wn,z):
    '''Get mks intensity assuming args are in simulation units
    
    :param np.ndarray A: vector potential in frequency domain with shape (w,r,phi,z)
    :param np.ndarray wn: frequency nodes
    :param int z: index into z slice of A'''
    E2 = np.fft.irfft(1j*wn[...,np.newaxis,np.newaxis]*A[...,z],axis=0)**2
    E2 *= (C.m_e*C.c**2/mks_length/C.e)**2
    return E2/377

def get_time_domain_power(intensity_mks,rn_mks):
    '''Get mks power assuming args are in simulation units, except for mks_length (meters)
    
    :param np.ndarray intensity: mks intensity with shape (t,r,phi)
    :param np.ndarray rn: radial nodes'''
    L2_mks = get_Lambda2(rn_mks)
    return np.sum(intensity_mks[:,:,0]*L2_mks[np.newaxis,:],axis=1)

def get_spectral_intensity(A,wn,z,C):
    '''Get I(w,r) or I(w,kr) in units determined by calibration factor `C`
    
    :param np.ndarray A: vector potential in w-r or w-kr space, with shape (w,r,phi,z)
    :param np.ndarray wn: frequency nodes
    :param int z: index into z slice of A
    :param np.ndarray C: calibration from `get_cal_factor`'''
    return C * np.abs(A[...,z]*wn[...,np.newaxis,np.newaxis])**2

def get_spectral_power(spectral_intensity,rn):
    '''Get P(w) in units determined by calibration factor `C` used to get `spectral_intensity`
    
    :param np.ndarray spectral_intensity: spectral intensity with shape (w,r,phi)
    :param np.ndarray rn: radial nodes'''
    L2 = get_Lambda2(rn)
    return np.sum(spectral_intensity[:,:,0]*L2[np.newaxis,:],axis=1)

(cl_refs,args) = init.setup_opencl(['rays.py','run','file=none','platform=cuda'])
cl_refs.add_program('../kernels','fft')

z00 = []
dz = []
k_icm = []
power = []

for file in sorted(total_list):
    A = np.load(file)
    s = file.split('_')
    data_ext_name = s[0]
    for word in s[1:-1]:
        data_ext_name += '_' + word
    data_ext_name += '_plot_ext.npy'
    data_ext = np.load(data_ext_name)
    dw,wc,tmax,wn,rn = get_freq_time_data(A,data_ext)
    k_icm.append(wn/(2*np.pi*mks_length*100))
    Nz = A.shape[3]
    dz.append(mks_length*(data_ext[7]-data_ext[6])/(Nz-1))
    z00.append(mks_length*data_ext[6])
    # Calibration of spectral power using initial condition
    I0 = get_time_domain_intensity(A,wn,0)
    P0 = get_time_domain_power(I0,rn*mks_length)
    U00 = np.sum(P0)*tmax*mks_length/C.c/P0.shape[0]
    dk_icm = k_icm[-1][1] - k_icm[-1][0]
    Coeff = get_cal_factor(A,wn,rn,U00,dk_icm)
    # Get spectral power at all z positions
    Pz = np.zeros((Nz,A.shape[0]))
    for k in range(Nz):
        Iw = get_spectral_intensity(A,wn,k,Coeff)
        Pz[k,:] = get_spectral_power(Iw,rn)
    power.append(np.copy(Pz))

plt.close('all')

In [ ]:
print('Pulse energy =',U00)
print(sorted(total_list))

In [ ]:
LWIR_energy = []
for irun in range(nruns):
    low_idx = 0
    high_idx = power[irun].shape[0]
    Nz = power[irun].shape[0]
    Uz = np.zeros(Nz)
    for k in range(Nz):
        Uz[k] = np.sum(power[irun][k,low_idx:high_idx])*(k_icm[irun][1]-k_icm[irun][0])
    LWIR_energy.append(Uz)

plt.figure(figsize=(4,6),dpi=200)
plt.subplot(211)
for irun in range(0,nruns):
    z = lambda i: '$z$ = {:.0f} cm'.format(-20 + i*4) 
    for zi in [2,3,4,5]:
        plt.plot(k_icm[irun],power[irun][zi,:]*1e9,label=z(zi))
plt.xlim(0,5000)
plt.ylim(0,1)
plt.xlabel('Wavenumber (cm$^{-1}$)')
plt.ylabel('Spectral Power (nJ$\cdot$cm)')
plt.legend(fontsize=10)

plt.subplot(212)
max_energy = 0.0
for irun in range(0,nruns):
    Nz = power[irun].shape[0]
    z = np.linspace(z00[irun],z00[irun]+dz[irun]*(Nz-1),Nz)
    plt.plot(z*100,LWIR_energy[irun]*1e9)
    max_energy = np.max([max_energy,np.max(LWIR_energy[irun])])
plt.xlabel('z (cm)')
plt.ylabel('Energy (nJ)')
plt.tight_layout()

In [ ]:
k = 4
tool = grid.HankelTransformTool(A.shape[1],rn[1]-rn[0],0,cl_refs,A.shape[1])
tool.AllocateDeviceMemory(A.shape[:3])
Awk = A.copy()
tool.kspace(Awk) # in 4D the argument is changed in place
kr = np.sqrt(tool.kr2() / (2 * np.pi * mks_length * 100)**2) # remember this is reversed
Iwk = get_spectral_intensity(Awk,wn,k,Coeff)
Iwk_loaded = (Iwk[:,:,0] * get_Lambda2(rn)[np.newaxis,:])[:,::-1]
plt.figure(1,dpi=200,figsize=(4,4))
plt.imshow(np.log10(1e-15+Iwk_loaded.swapaxes(0,1)),origin='lower',aspect='auto',extent=[0,k_icm[0][-1],0,kr.shape[0]],
           vmin=-12,vmax=-10,cmap='plasma')
plt.xlabel("Wavenumbers (cm$^{-1}$)")
plt.ylabel("Radial Mode Number")
plt.xlim(0,4000)
plt.ylim(0,60)
b = plt.colorbar(orientation='horizontal')
b.set_label("log$_{10}$Intensity(J$\cdot$cm/mode)")

print(dk_icm)
print('max loaded intensity =',np.max(Iwk_loaded))
print('total energy =',np.sum(Iwk_loaded)*dk_icm)
print('kr max =',kr[0])